### 1-Using Bootstrap to calculate the metric of flatten classification and hierarchical classification
    1 - Logistic Regression
    2 - Random Forest
    3 - HMNNF

In [18]:
from __future__ import absolute_import, division, print_function

import os
import re
import argparse
import joblib
import h5py
import time
import numpy as np
import pandas as pd
from scipy import interp
from datetime import datetime
import matplotlib.pyplot as plt

import keras
from keras import backend as K
from keras.backend import clear_session
from keras import regularizers 
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, InputLayer, Embedding, Reshape, Concatenate, Masking
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, confusion_matrix, roc_curve, auc, classification_report, precision_recall_curve, average_precision_score

from Data_helper import LoadDataTrial
from utils.layers import ExternalMasking
from utils.grud_layers import GRUD, Bidirectional_for_GRUD
from BuildModel import LoadModel
from utils.fusion import Gmu as Gmu
from utils.attention_function import attention_3d_block_spatial as PreAttentionSpatial
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [19]:
arg_parser = argparse.ArgumentParser()
arg_parser.add_argument('--working_path', default='.')

## data
arg_parser.add_argument('--dataset_name', default='phase_viii', 
                         help='The data files should be saved in [working_path]/data/[dataset_name] directory.')
arg_parser.add_argument('--fold', type=int, default=0, 
                         help='the fold data taken to use, there are 5 folds or 10 folds')
arg_parser.add_argument('--label', default=-1, type=int, choices=[-1,0,1,2,3,4],
                         help='the label type')

## model
arg_parser.add_argument('--model_type', default='HMM_LF', choices=['LR', 'RF', 'SVM', 'DNN', 'DNN_EE', 'GRUD', 'HMM_EF', 'HMM_LF'])
arg_parser.add_argument('--max_timestep', type=int, default=200, 
                        help='Time series of at most # time steps are used. Default: 200.')
arg_parser.add_argument('--max_timestamp', type=int, default=48*60*60,choices=[48*60*60, 72*60*60, 96*60*60, 120*60*60],
                        help='Time series of at most # seconds are used. Default: 48 (hours).')
arg_parser.add_argument('--use_bidirectional_rnn', default=False)
# Train
arg_parser.add_argument('--trainORvalidation', default='Train', choices=['Train', 'Validation'])
arg_parser.add_argument('--epochs', type=int, default=100)
arg_parser.add_argument('--early_stopping_patience', type=int, default=10)
arg_parser.add_argument('--batch_size', type=int, default=32)
# set the actual arguments if running in notebook
if not (__name__ == '__main__' and '__file__' in globals()):
    ARGS = arg_parser.parse_args([
        '--model_type', 'HMM_LF',
        '--dataset_name', 'phase_viii',
        '--fold', '0',
        '--epochs', '100',
        '--trainORvalidation', 'Train'
    ])
else:
      ARGS = arg_parser.parse_args()

print('Arguments:', ARGS)

Arguments: Namespace(batch_size=32, dataset_name='phase_viii', early_stopping_patience=10, epochs=100, fold=0, label=-1, max_timestamp=172800, max_timestep=200, model_type='HMM_LF', trainORvalidation='Train', use_bidirectional_rnn=False, working_path='.')


In [3]:
def plot_cm(types, model_info, y_true, y_pred, runtime):
    # get the confusion matrix
    C = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(C, columns=['0','1','2','3','4','5','6'])
    df_cm['bootstrap_index'] = runtime
    df_cm['label'] = types

    return df_cm
    
def save_report(types, model_info, y_true, y_pred, runtime):
    report = classification_report(y_true, y_pred,digits=5, output_dict = True)
    report = pd.DataFrame(report).transpose().reset_index()
    report['bootstrap_index'] = runtime
    report['label'] = types

    return report

In [4]:
if ARGS.model_type == 'LR':
    model_path = os.path.join(ARGS.working_path, 'model_tuning', 'phase_viii', '48hours', '20220604', 'LR_423dim_48hrs_Paper(optuna)')
elif ARGS.model_type == 'RF':
    model_path = os.path.join(ARGS.working_path, 'model_tuning', 'phase_viii', '48hours', '20220604', 'RF_423dim_48hrs_Paper(optuna)')
else:
    model_path = os.path.join(ARGS.working_path, 'model_tuning', 'phase_viii', '48hours', '20220604', 'HMMLF_CONCAT_423dim_48hrs_Paper(last)')

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
output_path = os.path.join(ARGS.working_path, 'test', 'phase_viii', '48hours', '20220604', 'Hierarchy_Test(last)')
if not os.path.exists(output_path):
    os.makedirs(output_path)

T = time.strftime("%Y%m%d%H%M%S", time.localtime())
LABEL_DICT = {'-1':'flatten multi-classification', 
              '0': 'infectious and non-infectious', 
              '1': 'bacterial, viral and others', 
              '2': 'NIID and tumor', 
              '3': 'AD and AID', 
              '4': 'HM and SM'}

# Load the data
dataset = LoadDataTrial(data_path=os.path.join('.', 'data', ARGS.dataset_name, '48hours', 'processed', 'raw/data4hc_v20220401'), 
                        model_type=ARGS.model_type,
                        label_name=ARGS.label,
                        max_timestep=ARGS.max_timestep,
                        max_timestamp=ARGS.max_timestamp)
X_train, y_train_1d, nclasses_train, folds_train, shapes_train = dataset.training_generator(ARGS.fold)
X_validation, y_validation_1d, nclasses_validation,folds_validation, shapes_validation = dataset.validation_generator(ARGS.fold)
X_test, y_test_1d, nclasses_test, folds_test, shapes_test = dataset.test_generator(ARGS.fold)

y_train = to_categorical(y_train_1d, num_classes=nclasses_train)           
y_validation = to_categorical(y_validation_1d, num_classes=nclasses_train)
y_test = to_categorical(y_test_1d, num_classes=nclasses_train)

# Load the model
if ARGS.model_type == 'HMM_LF':
    model_f = LoadModel(os.path.join(model_path, 'HMMLF(flatten multi-classification).h5'))
    model_0 = LoadModel(os.path.join(model_path, 'HMMLF(infectious and non-infectious).h5'))
    model_1 = LoadModel(os.path.join(model_path, 'HMMLF(bacterial, viral and others).h5'))
    model_2 = LoadModel(os.path.join(model_path, 'HMMLF(NIID and tumor).h5'))
    model_3 = LoadModel(os.path.join(model_path, 'HMMLF(AD and AID).h5'))
    model_4 = LoadModel(os.path.join(model_path, 'HMMLF(HM and SM).h5'))
elif ARGS.model_type == 'LR':
    model_f = joblib.load(os.path.join(model_path, 'LR(flatten multi-classification).pkl'))
    model_0 = joblib.load(os.path.join(model_path, 'LR(infectious and non-infectious).pkl'))
    model_1 = joblib.load(os.path.join(model_path, 'LR(bacterial, viral and others).pkl'))
    model_2 = joblib.load(os.path.join(model_path, 'LR(NIID and tumor).pkl'))
    model_3 = joblib.load(os.path.join(model_path, 'LR(AD and AID).pkl'))
    model_4 = joblib.load(os.path.join(model_path, 'LR(HM and SM).pkl'))
else:
    model_f = joblib.load(os.path.join(model_path, 'RF(flatten multi-classification).pkl'))
    model_0 = joblib.load(os.path.join(model_path, 'RF(infectious and non-infectious).pkl'))
    model_1 = joblib.load(os.path.join(model_path, 'RF(bacterial, viral and others).pkl'))
    model_2 = joblib.load(os.path.join(model_path, 'RF(NIID and tumor).pkl'))
    model_3 = joblib.load(os.path.join(model_path, 'RF(AD and AID).pkl'))
    model_4 = joblib.load(os.path.join(model_path, 'RF(HM and SM).pkl'))


/home/zju/anaconda3/envs/gru/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
# set bootstrap on test dataset
df_cms_f = pd.DataFrame(columns=['bootstrap_index', 'label', '0','1','2','3','4','5','6'])
df_cms_h = pd.DataFrame(columns=['bootstrap_index', 'label', '0','1','2','3','4','5','6'])
df_reports_f = pd.DataFrame(columns=['bootstrap_index', 'label', 'index', 'precision', 'recall', 'f1-score', 'support'])
df_reports_h = pd.DataFrame(columns=['bootstrap_index', 'label', 'index', 'precision', 'recall', 'f1-score', 'support'])
# predict the test samples
if ARGS.model_type == 'HMM_LF':
    for runtime in range(1000):
        print("[Info] The {}-th iteration is processing!".format(runtime))
        idx = np.random.randint(0, len(X_test[0]) - 1, size=len(X_test[0]))
        X_test_boot = [x[idx] for x in X_test]
        y_test_boot = y_test_1d[idx]
        folds = folds_test[idx]
        
        y_pred = []
        for s in range(len(y_test_boot)):
            # infectious OR Noninfectious
            y_pred_proba_0 = model_0.predict([np.expand_dims(x[s], axis=0) for x in X_test_boot])
            y_pred_0 = np.argmax(y_pred_proba_0, axis=1)

            if y_pred_0[0] == 0:
                # bacterial, viral OR fungal
                y_pred_proba_1 = model_1.predict([np.expand_dims(x[s], axis=0) for x in X_test_boot])
                y_pred_1 = np.argmax(y_pred_proba_1, axis=1)
                if y_pred_1[0] == 0:
                    y_pred.append(0)
                elif y_pred_1[0] == 1:
                    y_pred.append(1)
                elif y_pred_1[0] == 2:
                    y_pred.append(2)
                    
            elif y_pred_0[0] == 1:
                # NIID OR Neoplastic
                y_pred_proba_2 = model_2.predict([np.expand_dims(x[s], axis=0) for x in X_test_boot])
                y_pred_2 = np.argmax(y_pred_proba_2, axis=1)
                if y_pred_2[0] == 0:
                    # AID OR AIFD
                    y_pred_proba_3 = model_3.predict([np.expand_dims(x[s], axis=0) for x in X_test_boot])
                    y_pred_3 = np.argmax(y_pred_proba_3, axis=1)
                    if y_pred_3[0] == 0:
                        y_pred.append(3)
                    elif y_pred_3[0] == 1:
                        y_pred.append(4)
                elif y_pred_2[0] == 1:
                    # HM OR SM
                    y_pred_proba_4 = model_4.predict([np.expand_dims(x[s], axis=0) for x in X_test_boot])
                    y_pred_4 = np.argmax(y_pred_proba_4, axis=1)
                    if y_pred_4[0] == 0:
                        y_pred.append(5)
                    elif y_pred_4[0] == 1:
                        y_pred.append(6)
        
        # predict respectively
        y_pred_proba_f = model_f.predict(X_test_boot)
        # Process the predicted label of hierarchy1
        y_pred_f = np.argmax(y_pred_proba_f, axis=1)

        
        # save the confusion matrix
        df_cm_h = plot_cm('Hierarchy', ARGS.model_type, y_test_boot.tolist(), y_pred, runtime)
        df_cms_h = pd.concat([df_cms_h, df_cm_h], axis=0, ignore_index=True)
        
        df_cm_f = plot_cm('Flatten', ARGS.model_type, y_test_boot.tolist(), y_pred_f.tolist(), runtime)
        df_cms_f = pd.concat([df_cms_f, df_cm_f], axis=0, ignore_index=True)
        
        # save the reports
        df_report_h = save_report('Hierarchy', ARGS.model_type, y_test_boot.tolist(), y_pred, runtime)
        df_reports_h = pd.concat([df_reports_h, df_report_h], axis=0, ignore_index=True)
        
        df_report_f = save_report('Flatten', ARGS.model_type, y_test_boot.tolist(),  y_pred_f.tolist(), runtime)
        df_reports_f = pd.concat([df_reports_f, df_report_f], axis=0, ignore_index=True)

    df_cms_h.to_csv(os.path.join(output_path, 'confusion_matrix(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Hierarchy')))
    df_cms_f.to_csv(os.path.join(output_path, 'confusion_matrix(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Flatten')))
    df_reports_h.to_csv(os.path.join(output_path, 'classification_reuslts(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Hierarchy')))
    df_reports_f.to_csv(os.path.join(output_path, 'classification_reuslts(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Flatten')))     
        
else:
    for runtime in range(1000):
        print("[Info] The {}-th iteration is processing!".format(runtime))
        idx = np.random.randint(0, len(X_test) - 1, size=len(X_test))
        X_test_boot = X_test[idx]
        y_test_boot = y_test_1d[idx]
        folds = folds_test[idx]
        
        y_pred = []
        for s in range(len(y_test_boot)):
            # infectious OR Noninfectious
            y_pred_0 = model_0.predict(np.expand_dims(X_test_boot[s], axis=0))

            if y_pred_0[0] == 0:
                # bacterial, viral OR fungal
                y_pred_1 = model_1.predict(np.expand_dims(X_test_boot[s], axis=0))
                if y_pred_1[0] == 0:
                    y_pred.append(0)
                elif y_pred_1[0] == 1:
                    y_pred.append(1)
                elif y_pred_1[0] == 2:
                    y_pred.append(2)
                    
            elif y_pred_0[0] == 1:
                # NIID OR Neoplastic
                y_pred_2 = model_2.predict(np.expand_dims(X_test_boot[s], axis=0))
                if y_pred_2[0] == 0:
                    # AID OR AIFD
                    y_pred_3 = model_3.predict(np.expand_dims(X_test_boot[s], axis=0))
                    if y_pred_3[0] == 0:
                        y_pred.append(3)
                    elif y_pred_3[0] == 1:
                        y_pred.append(4)
                elif y_pred_2[0] == 1:
                    # HM OR SM
                    y_pred_4 = model_4.predict(np.expand_dims(X_test_boot[s], axis=0))
                    if y_pred_4[0] == 0:
                        y_pred.append(5)
                    elif y_pred_4[0] == 1:
                        y_pred.append(6)
        # predict respectively
        y_pred_f = model_f.predict(X_test_boot)

        # save the confusion matrix
        df_cm_h = plot_cm('Hierarchy', ARGS.model_type, y_test_boot.tolist(), y_pred, runtime)
        df_cms_h = pd.concat([df_cms_h, df_cm_h], axis=0, ignore_index=True)
        
        df_cm_f = plot_cm('Flatten', ARGS.model_type, y_test_boot.tolist(), y_pred_f.tolist(), runtime)
        df_cms_f = pd.concat([df_cms_f, df_cm_f], axis=0, ignore_index=True)
        
        # save the reports
        df_report_h = save_report('Hierarchy', ARGS.model_type, y_test_boot.tolist(), y_pred, runtime)
        df_reports_h = pd.concat([df_reports_h, df_report_h], axis=0, ignore_index=True)
        
        df_report_f = save_report('Flatten', ARGS.model_type, y_test_boot.tolist(),  y_pred_f.tolist(), runtime)
        df_reports_f = pd.concat([df_reports_f, df_report_f], axis=0, ignore_index=True)

    df_cms_h.to_csv(os.path.join(output_path, 'confusion_matrix(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Hierarchy')))
    df_cms_f.to_csv(os.path.join(output_path, 'confusion_matrix(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Flatten')))
    df_reports_h.to_csv(os.path.join(output_path, 'classification_reuslts(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Hierarchy')))
    df_reports_f.to_csv(os.path.join(output_path, 'classification_reuslts(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Flatten')))   

[Info] The 0-th iteration is processing!
[Info] The 1-th iteration is processing!
[Info] The 2-th iteration is processing!
[Info] The 3-th iteration is processing!
[Info] The 4-th iteration is processing!
[Info] The 5-th iteration is processing!
[Info] The 6-th iteration is processing!
[Info] The 7-th iteration is processing!
[Info] The 8-th iteration is processing!
[Info] The 9-th iteration is processing!
[Info] The 10-th iteration is processing!
[Info] The 11-th iteration is processing!
[Info] The 12-th iteration is processing!
[Info] The 13-th iteration is processing!
[Info] The 14-th iteration is processing!
[Info] The 15-th iteration is processing!
[Info] The 16-th iteration is processing!
[Info] The 17-th iteration is processing!
[Info] The 18-th iteration is processing!
[Info] The 19-th iteration is processing!
[Info] The 20-th iteration is processing!
[Info] The 21-th iteration is processing!
[Info] The 22-th iteration is processing!
[Info] The 23-th iteration is processing!
[I

[Info] The 194-th iteration is processing!
[Info] The 195-th iteration is processing!
[Info] The 196-th iteration is processing!
[Info] The 197-th iteration is processing!
[Info] The 198-th iteration is processing!
[Info] The 199-th iteration is processing!
[Info] The 200-th iteration is processing!
[Info] The 201-th iteration is processing!
[Info] The 202-th iteration is processing!
[Info] The 203-th iteration is processing!
[Info] The 204-th iteration is processing!
[Info] The 205-th iteration is processing!
[Info] The 206-th iteration is processing!
[Info] The 207-th iteration is processing!
[Info] The 208-th iteration is processing!
[Info] The 209-th iteration is processing!
[Info] The 210-th iteration is processing!
[Info] The 211-th iteration is processing!
[Info] The 212-th iteration is processing!
[Info] The 213-th iteration is processing!
[Info] The 214-th iteration is processing!
[Info] The 215-th iteration is processing!
[Info] The 216-th iteration is processing!
[Info] The 

[Info] The 385-th iteration is processing!
[Info] The 386-th iteration is processing!
[Info] The 387-th iteration is processing!
[Info] The 388-th iteration is processing!
[Info] The 389-th iteration is processing!
[Info] The 390-th iteration is processing!
[Info] The 391-th iteration is processing!
[Info] The 392-th iteration is processing!
[Info] The 393-th iteration is processing!
[Info] The 394-th iteration is processing!
[Info] The 395-th iteration is processing!
[Info] The 396-th iteration is processing!
[Info] The 397-th iteration is processing!
[Info] The 398-th iteration is processing!
[Info] The 399-th iteration is processing!
[Info] The 400-th iteration is processing!
[Info] The 401-th iteration is processing!
[Info] The 402-th iteration is processing!
[Info] The 403-th iteration is processing!
[Info] The 404-th iteration is processing!
[Info] The 405-th iteration is processing!
[Info] The 406-th iteration is processing!
[Info] The 407-th iteration is processing!
[Info] The 

[Info] The 576-th iteration is processing!
[Info] The 577-th iteration is processing!
[Info] The 578-th iteration is processing!
[Info] The 579-th iteration is processing!
[Info] The 580-th iteration is processing!
[Info] The 581-th iteration is processing!
[Info] The 582-th iteration is processing!
[Info] The 583-th iteration is processing!
[Info] The 584-th iteration is processing!
[Info] The 585-th iteration is processing!
[Info] The 586-th iteration is processing!
[Info] The 587-th iteration is processing!
[Info] The 588-th iteration is processing!
[Info] The 589-th iteration is processing!
[Info] The 590-th iteration is processing!
[Info] The 591-th iteration is processing!
[Info] The 592-th iteration is processing!
[Info] The 593-th iteration is processing!
[Info] The 594-th iteration is processing!
[Info] The 595-th iteration is processing!
[Info] The 596-th iteration is processing!
[Info] The 597-th iteration is processing!
[Info] The 598-th iteration is processing!
[Info] The 

[Info] The 767-th iteration is processing!
[Info] The 768-th iteration is processing!
[Info] The 769-th iteration is processing!
[Info] The 770-th iteration is processing!
[Info] The 771-th iteration is processing!
[Info] The 772-th iteration is processing!
[Info] The 773-th iteration is processing!
[Info] The 774-th iteration is processing!
[Info] The 775-th iteration is processing!
[Info] The 776-th iteration is processing!
[Info] The 777-th iteration is processing!
[Info] The 778-th iteration is processing!
[Info] The 779-th iteration is processing!
[Info] The 780-th iteration is processing!
[Info] The 781-th iteration is processing!
[Info] The 782-th iteration is processing!
[Info] The 783-th iteration is processing!
[Info] The 784-th iteration is processing!
[Info] The 785-th iteration is processing!
[Info] The 786-th iteration is processing!
[Info] The 787-th iteration is processing!
[Info] The 788-th iteration is processing!
[Info] The 789-th iteration is processing!
[Info] The 

[Info] The 958-th iteration is processing!
[Info] The 959-th iteration is processing!
[Info] The 960-th iteration is processing!
[Info] The 961-th iteration is processing!
[Info] The 962-th iteration is processing!
[Info] The 963-th iteration is processing!
[Info] The 964-th iteration is processing!
[Info] The 965-th iteration is processing!
[Info] The 966-th iteration is processing!
[Info] The 967-th iteration is processing!
[Info] The 968-th iteration is processing!
[Info] The 969-th iteration is processing!
[Info] The 970-th iteration is processing!
[Info] The 971-th iteration is processing!
[Info] The 972-th iteration is processing!
[Info] The 973-th iteration is processing!
[Info] The 974-th iteration is processing!
[Info] The 975-th iteration is processing!
[Info] The 976-th iteration is processing!
[Info] The 977-th iteration is processing!
[Info] The 978-th iteration is processing!
[Info] The 979-th iteration is processing!
[Info] The 980-th iteration is processing!
[Info] The 

### 2 - To get the statistics of bootstraping results

In [6]:
df_h = pd.read_csv(os.path.join(output_path, 'classification_reuslts(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Hierarchy')))
df_f = pd.read_csv(os.path.join(output_path, 'classification_reuslts(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Flatten')))

In [7]:
# Get the summary data of flatten classification
accuracy_f = df_f.loc[df_f['index']=='accuracy', :]['precision'].values
macro_precision_f = df_f.loc[df_f['index'] == 'macro avg', :]['precision'].values
macro_recall_f = df_f.loc[df_f['index'] == 'macro avg', :]['recall'].values
macro_f1_f = df_f.loc[df_f['index'] == 'macro avg', :]['f1-score'].values

summary = {'accuracy': {'mean': accuracy_f.mean(), 'std': accuracy_f.std()},
           'macro_precison':{'mean': macro_precision_f.mean(), 'std': macro_precision_f.std()},
           'macro_recall': {'mean': macro_recall_f.mean(), 'std': macro_recall_f.std()},
           'macro_f1': {'mean': macro_f1_f.mean(), 'std': macro_f1_f.std()}}
df_sum = pd.DataFrame(summary)
df_sum.to_csv(os.path.join(output_path, 'Summary(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Flatten')))
# df_sum

In [8]:
# Get the summary data of flatten classification
accuracy_h = df_h.loc[df_h['index']=='accuracy', :]['precision'].values
macro_precision_h = df_h.loc[df_h['index'] == 'macro avg', :]['precision'].values
macro_recall_h = df_h.loc[df_h['index'] == 'macro avg', :]['recall'].values
macro_f1_h = df_h.loc[df_h['index'] == 'macro avg', :]['f1-score'].values

summary = {'accuracy': {'mean': accuracy_h.mean(), 'std': accuracy_h.std()},
           'macro_precison':{'mean': macro_precision_h.mean(), 'std': macro_precision_h.std()},
           'macro_recall': {'mean': macro_recall_h.mean(), 'std': macro_recall_h.std()},
           'macro_f1': {'mean': macro_f1_h.mean(), 'std': macro_f1_h.std()}}
df_sum = pd.DataFrame(summary)
df_sum.to_csv(os.path.join(output_path, 'Summary(BTSP)({0})({1}).csv'.format(ARGS.model_type, 'Hierarchy')))
# df_sum

In [20]:
data = pd.read_csv("/mnt/data/wzx/jupyter_notebook/HC4FUO/test/phase_viii/48hours/20220604/Hierarchy_Test(last)/Summary(BTSP)(LR)(Flatten).csv")
data

,Unnamed: 0,accuracy,macro_precison,macro_recall,macro_f1
0,mean,0.609591,0.604936,0.308903,0.339009
1,std,0.010590,0.053059,0.010282,0.013743


In [17]:
data = pd.read_csv("/mnt/data/wzx/jupyter_notebook/HC4FUO/test/phase_viii/48hours/20220604/Hierarchy_Test(last)/Summary(BTSP)(LR)(Hierarchy).csv")
data

,Unnamed: 0,accuracy,macro_precison,macro_recall,macro_f1
0,mean,0.621161,0.491936,0.371053,0.391408
1,std,0.010300,0.023500,0.011994,0.013489
